# Języki Programowania Python i R


## dr inż. Patryk Jasik
### Division of Theoretical Physics and Quantum Information
### Institute of Physics and Computer Science
### Faculty of Applied Physics and Mathematics
### Gdansk University of Technology

# scikit-learn docs
## https://scikit-learn.org/stable/

In [ ]:
#%config Completer.use_jedi = False

**Regression** - is an approach for modelling the relationship between a scalar response and one or more explanatory variables (also known as dependent and independent variables).

2D problem\
$$
y_i = a*x_i + b
$$


Multidimensional problem\
$$
y_i = a_1*x_{i1} + a_2*x_{i2} + ... + a_p*x_{ip} + intercept
$$

In [ ]:
#loading the necessary packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
#measurements of physical and chemical properties of Portuguese Vinho Verde wines (white and red) 
wine = pd.read_csv("data/winequality-all.csv", comment="#")
wine.head()

In [ ]:
#helpful information about the dataset that can be saved as variables
wine.shape

In [ ]:
wine.columns

In [ ]:
wine.info()

In [ ]:
#the color of the wine is of type object, so we need to change this variable to a categorical one
wine.color = wine.color.astype("category")
wine.info()

In [ ]:
wine

In [ ]:
#basic statistics
wine.describe()

In [ ]:
pd.DataFrame(wine['free.sulfur.dioxide']).boxplot()

In [ ]:
pd.DataFrame(wine['free.sulfur.dioxide']).hist()

In [ ]:
wine.describe(include='all')

### goal - we will check whether alcohol is a function of the remaining 10 variables and what is the relationship.
### Thanks to this, we will be able to explain the derivative of what set of factors the given alcohol content is, as well as predict the alcohol content in the newly produced batch of wine.

In [ ]:
#we check how many red wines and how many white wines are in our collection
wine.color.value_counts()

In [ ]:
#We check how the statistics for red and white wines differ

#creation of the subsets
white_wine = wine[wine.color == 'white']
red_wine = wine[wine.color == 'red']

In [ ]:
white_wine.describe()

In [ ]:
red_wine.describe()

In [ ]:
#the target variable
y = white_wine.iloc[:, -3]
y.head(10)

In [ ]:
y.tail(10)

In [ ]:
#predictors
X = white_wine.iloc[:, :-3]
X.head()

In [ ]:
X.shape

# Correlation Study

An important element in statistical analysis is the study of the correlation of variables. It is thanks to it that we know if there is any relationship between the data sets.

It is very important to investigate the correlation when building a regression model. By examining the correlation of our goal (in this example 'alcohol') with the features, we can find out which of the featues is the most important and which will not help us create the appropriate model.

The most frequently studied correlations are:
 - <a href = https://en.wikipedia.org/wiki/Pearson_correlation_coefficient> **Pearson** </a> - examines the linear relationship in the X and Y data
 
$$
r_{p}= \frac{cov(X,Y)}{\sigma_X \sigma_Y}
$$

 - <a href = https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient> **Spearman** </a> - examines the rank dependence of X and Y data. A high value of the correlation coefficient does not mean that the data is linearly correlated, but as the value of X increases, the values of Y increase, but not necessarily always by the same value
 
 $$
 r_s = \frac{cov(R(X)R(Y))}{\sigma_{R(X)}\sigma_{R(Y)}}
 $$
 
*where*:
 
 $cov()$ - covariance, 
 
 $\sigma$ - standard deviation
 
 $R()$ - data rank

In [ ]:
#let's calculate the Pearson's linear correlation coefficients
corr_P = white_wine.corr("pearson")
corr_P.shape

In [ ]:
corr_P

In [ ]:
#we create a triangular matrix and display a correlation coefficients greater than 0.4
corr_P_tri = corr_P.where(np.triu(np.ones(corr_P.shape, dtype=bool), k=1)).stack().sort_values()
corr_P_tri

In [ ]:
corr_P_tri[abs(corr_P_tri)>0.4]

In [ ]:
#visualization of correlation using seaborn pairplot figure
sns.pairplot(white_wine)
plt.show()

In [ ]:
#let's calculate the Spearman's correlation coefficients
corr_S = white_wine.corr("spearman")
corr_S.shape

In [ ]:
corr_S

In [ ]:
#we create a triangular matrix and display a correlation coefficients greater than 0.4
corr_S_tri = corr_S.where(np.triu(np.ones(corr_S.shape, dtype=bool), k=1)).stack().sort_values()
corr_S_tri[abs(corr_S_tri)>0.4]

In [ ]:
#we create a linear regression model
import sklearn.linear_model
lm = sklearn.linear_model.LinearRegression()

In [ ]:
lm

In [ ]:
help(sklearn.linear_model.LinearRegression())

In [ ]:
lm

In [ ]:
#let's train linear model
lm.fit(X,y)

In [ ]:
#coefficients of the linear model
lm.coef_

In [ ]:
X.columns

In [ ]:
#intercept
lm.intercept_

In [ ]:
X.describe()

In [ ]:
#the mean value of the target variable
y.mean()

In [ ]:
x_new = X.mean().values.reshape(1,-1)
x_new

In [ ]:
#let's do prediction for the mean value of all predictors
lm.predict(x_new+0.001)

In [ ]:
#for a standardized dataset, the regression coefficients gain a useful interpretation
#i.e. the greater the value of the coefficient's modulus,
#the more significant it has an impact on the value of the objective variable
X_std = (X-X.mean(axis=0))/X.std(axis=0) # axis=0 => columns
X_std.describe()

In [ ]:
lm_std = sklearn.linear_model.LinearRegression()
lm_std.fit(X_std, (y-y.mean())/y.std())

In [ ]:
pd.Series(np.abs(lm_std.coef_), index=X.columns.to_list()).round(4).sort_values(ascending=False)

In [ ]:
# assessment of the quality of the model
# compare the fitted values calculated by the model with the original values
y_pred = lm.predict(X)
y_pred[0:15]

In [ ]:
y[0:15]

In [ ]:
#R2 - determination coefficient
lm.score(X,y)

In [ ]:
sklearn.metrics.r2_score(y,y_pred)

In [ ]:
#MAE
sklearn.metrics.mean_absolute_error(y,y_pred)

In [ ]:
#MSE
sklearn.metrics.mean_squared_error(y,y_pred)

In [ ]:
#RMSE
sklearn.metrics.mean_squared_error(y,y_pred, squared=False)

In [ ]:
help(sklearn.metrics.mean_squared_error)

In [ ]:
#MAPE
sklearn.metrics.mean_absolute_percentage_error(y,y_pred)*100

In [ ]:
# we care about the model's good predictive ability
#but we also make sure not to overfit the model,
#so we divide the dataset into a training dataset (80%) and a test dataset (20%)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,
                                                                        y,
                                                                        test_size=0.2,
                                                                        random_state=12345)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#we will create a function that fits the linear regression model to a given sample
#and computes errors of prediction
def fit_regression(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    r2 = sklearn.metrics.r2_score
    rmse = sklearn.metrics.mean_squared_error
    mae = sklearn.metrics.mean_absolute_error
    
    return {
        "r_score_tr": r2(y_train, y_train_pred),
        "r_score_te": r2(y_test, y_test_pred),
        "RMSE_tr": rmse(y_train, y_train_pred, squared=False),
        "RMSE_te": rmse(y_test, y_test_pred, squared=False),
        "MAE_tr": mae(y_train, y_train_pred),
        "MAE_te": mae(y_test, y_test_pred)
    }

In [ ]:
#operation of the above function and results
params = ["Lin. Reg."]
res = [fit_regression(sklearn.linear_model.LinearRegression(),
                      X_train,
                      X_test,
                      y_train,
                      y_test)]

results = pd.DataFrame(res, index=params)

In [ ]:
results

In [ ]:
#let's change the random state
X_train_rs123, X_test_rs123, y_train_rs123, y_test_rs123 = sklearn.model_selection.train_test_split(
                                                                        X,
                                                                        y,
                                                                        test_size=0.2,
                                                                        random_state=123)

In [ ]:
params.append("Lin. Reg. rs123")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X_train_rs123, X_test_rs123, y_train_rs123, y_test_rs123))
results = pd.DataFrame(res, index=params)

In [ ]:
results

In [ ]:
#let's change the size of the test dataset
X_train_70, X_test_70, y_train_70, y_test_70 = sklearn.model_selection.train_test_split(
                                                                        X,
                                                                        y,
                                                                        test_size=0.3,
                                                                        random_state=12345)

In [ ]:
params.append("Lin. Reg. test size 70")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X_train_70, X_test_70, y_train_70, y_test_70))
results = pd.DataFrame(res, index=params)

In [ ]:
results

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf.get_n_splits(X)

In [ ]:
kf

In [ ]:
X.reset_index(drop=True)

In [ ]:
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_kf, X_test_kf = X.iloc[train_index,:], X.iloc[test_index,:]
    y_train_kf, y_test_kf = y.iloc[train_index], y.iloc[test_index]
    display(X_train_kf)
    display(y_train_kf)
    display(X_test_kf)
    display(y_test_kf)

In [ ]:
#let's create a set of regression models
#we start with one model and we will add other models later on
methods = pd.Series({
    "lin_reg": sklearn.linear_model.LinearRegression()
})

In [ ]:
#evaluation function
def eval_function(X_train, X_test, y_train, y_test):
    cv_models = pd.concat([
        pd.Series(fit_regression(alg,
                                 X_train, X_test, y_train, y_test)) for alg in methods], axis=1).T
    cv_models.index = methods.index
    return cv_models

In [ ]:
#application of the evaluation function
#results summarizing the cross validation
from sklearn.model_selection import KFold

n_folds = 5

results_cv = [eval_function(X.iloc[train,:],
                            X.iloc[test,:],
                            y.iloc[train],
                            y.iloc[test]) for train, test in kf.split(X)]

sum(results_cv)/n_folds

In [ ]:
results_cv

In [ ]:
(sum(results_cv)/n_folds).to_dict()

In [ ]:
#we append the cross validation results to our main dataframe with all results
params.append(" ")
res.append( )
results = pd.DataFrame(res, index=params)

In [ ]:
results

In [ ]:
#cross validation could be done in a simpler way
from sklearn.model_selection import cross_validate

#scoring parameters 
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

cv_results = cross_validate(sklearn.linear_model.LinearRegression(),
                            X,
                            y,
                            scoring=('r2', 'neg_root_mean_squared_error', 'neg_mean_absolute_error'),
                            cv=5,
                            return_train_score=True)

In [ ]:
cv_results

In [ ]:
print("r_score_tr:", cv_results['train_r2'].mean().round(6))
print("r_score_te:", cv_results['test_r2'].mean().round(6))
print("RMSE_tr:", (cv_results['train_neg_root_mean_squared_error']*-1).mean().round(6))
print("RMSE_te:", (cv_results['test_neg_root_mean_squared_error']*-1).mean().round(6))
print("MAE_tr:", (cv_results['train_neg_mean_absolute_error']*-1).mean().round(6))
print("MAE_te:", (cv_results['test_neg_mean_absolute_error']*-1).mean().round(6))

## Introduction to statistical tests

<div class="alert alert-block alert-success">
   <b> Definitions </b>
    
**Statistical test** - a mathematical formula that allows you to estimate the probability of meeting a certain statistical hypothesis in a population based on a random sample from that population.
    
$\newline$

**Statistical hypothesis** - any assumption about the distribution of the population

$\newline$

**Null hypothesis $𝐻_0$** - Supposition we want to check with statistical tests.

$\newline$

**P-value** - the cumulative probability of drawing a sample of the same or more extreme as observed, assuming that the null hypothesis is true

</div>

<div class="alert alert-block alert-warning">
A certain threshold (<b>significance level</b> $\alpha$) is set when using statistical tests. If the P-value is less than this level, the null hypothesis can be rejected. Usually $\alpha$ = 0.05
</div>

<div class="alert alert-block alert-info">
    
Using P-value and significance level, we can reject the null hypothesis. However, this does not mean that we confirm the alternative hypothesis ($H_1$).
</div>

Using statistical tests, we can also check whether the data has a specific distribution.

So let's check if **pH** follow a normal distribution. We can do this with the <a href = https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.normaltest.html> *scipy.stats.normaltest()* </a> method.

The null hypothesis in this method is that the data is normally distributed.

Let us assume the significance level of $\alpha$ = 0.05. We can reject this hypothesis and say that **pH** is not normally distributed.

In [ ]:
from scipy.stats import normaltest, anderson

In [ ]:
white_wine.columns

In [ ]:
normaltest(white_wine['pH'])

In [ ]:
plt.hist(white_wine['pH'], bins = 20)

In [ ]:
#example of normal distribution
np.random.seed(42)
norm = [np.random.normal() for i in range(100000)]

In [ ]:
plt.hist(norm, bins = 20)

In [ ]:
normaltest(norm)

Let's check if the **pH** has a different distribution.

A method for checking other common distributions is <a href = https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson.html> *scipy.stats.anderson()* </a>

With its help, we can check such distributions as:

- norm - normal distribution
- expon - exponential distribution
- logistic - logistic distribution
- gumbel - Gumbel distribution

In [ ]:
for test in ['norm', 'expon', 'logistic', 'gumbel']:
    print('\nWe check whether the distribution is ', test)
    print( anderson(white_wine['pH'], dist = test))

So, the **pH** distribution does not belong to any of the above-mentioned.

## Outliers analysis

In [ ]:
#let's remove from the dataframe the categorical feature: color
white_wine = white_wine.iloc[:,:-1].copy()

In [ ]:
white_wine

## Z-score method
## https://en.wikipedia.org/wiki/Standard_score

In [ ]:
import scipy as sc

z_scores = sc.stats.zscore(white_wine)
z_scores

In [ ]:
abs_z_scores = np.abs(z_scores)
filtered_z_scores = (abs_z_scores < 3.5).all(axis=1) #kind of mask

In [ ]:
filtered_z_scores

In [ ]:
white_wine_wout_outl = white_wine[filtered_z_scores]

In [ ]:
#the dataframe without outliers
white_wine_wout_outl

In [ ]:
white_wine.shape

In [ ]:
# % of rejected observations
np.round(((3961 - 3765)/3961)*100, 1)

In [ ]:
white_wine.describe()

In [ ]:
white_wine_wout_outl.describe()

In [ ]:
# pairplot after removing outliers
sns.pairplot(white_wine_wout_outl)
plt.show()

In [ ]:
#let's look on the correlations after removing outliers
corr_P_wout_outl = white_wine_wout_outl.corr("pearson")

corr_P_tri_wout_outl = corr_P_wout_outl.where(np.triu(np.ones(corr_P_wout_outl.shape, dtype=bool), k=1)).stack().sort_values()

corr_P_tri_wout_outl[abs(corr_P_tri_wout_outl)>0.4]

In [ ]:
# now, we will create the new reg model based on dataset without outliers
X_wout_out = white_wine_wout_outl.iloc[:,:-2]

In [ ]:
X_wout_out

In [ ]:
y_wout_out = white_wine_wout_outl.iloc[:,-2]

In [ ]:
y_wout_out

In [ ]:
X_train_wo, X_test_wo, y_train_wo, y_test_wo = sklearn.model_selection.train_test_split(X_wout_out,
                                                                        y_wout_out,
                                                                        test_size=0.2,
                                                                        random_state=12345)

In [ ]:
params.append("Lin. Reg. wout out")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X_train_wo, X_test_wo, y_train_wo, y_test_wo))
results = pd.DataFrame(res, index=params)

In [ ]:
#list with models' labels
params

In [ ]:
#list of dictionaries with models' metrics
res

In [ ]:
#dictionary with metric form cross validation
(sum(results_cv)/n_folds).to_dict()

In [ ]:
#we append the cross validation results to our main dataframe with all results
params.append("Lin. Reg. cv")
res.append({'r_score_tr': 0.8586248173163549,
 'r_score_te': 0.8108207931849465,
 'RMSE_tr': 0.4533127781833432,
 'RMSE_te': 0.4716703237761769,
 'MAE_tr': 0.3009219857242825,
 'MAE_te': 0.31508307320096673})
results = pd.DataFrame(res, index=params)

In [ ]:
results

In [ ]:
# cross validation for dataset without outliers
methods = pd.Series({
    "lin_reg cv": sklearn.linear_model.LinearRegression()
})

kf = KFold(n_splits=5)

n_folds = 5

results_cv = [eval_function(X_wout_out.iloc[train,:],
                            X_wout_out.iloc[test,:],
                            y_wout_out.iloc[train],
                            y_wout_out.iloc[test]) for train, test in kf.split(X_wout_out)]

sum(results_cv)/n_folds

In [ ]:
results_cv

In [ ]:
#dictionary with metric form cross validation
(sum(results_cv)/n_folds).to_dict()

In [ ]:
params.append("Lin. Reg. wout outl cv")
res.append({'r_score_tr': 0.9132987305047608,
 'r_score_te': 0.8960650144709106,
 'RMSE_tr': 0.35508883398984054,
 'RMSE_te': 0.36592675298535493,
 'MAE_tr': 0.27634804874826024,
 'MAE_te': 0.28516400829616273})
results = pd.DataFrame(res, index=params)

In [ ]:
results

## Standarization and normalization of data

In [ ]:
# we will work with dataset without outliers
X_wout_out.describe()

In [ ]:
y_wout_out.describe()

In [ ]:
# let's calculate the mean values
X_wout_out_mean = X_wout_out.mean()
X_wout_out_mean

In [ ]:
# and standard deviation
X_wout_out_std = X_wout_out.std()

In [ ]:
# standarization of the predictors
X_wo_std = (X_wout_out - X_wout_out_mean)/X_wout_out_std

In [ ]:
X_wo_std.describe()

In [ ]:
# and the same we will do with target variable
y_wo_m = y_wout_out.mean()
y_wo_sd = y_wout_out.std()

In [ ]:
y_wo_std = (y_wout_out-y_wo_m)/y_wo_sd

In [ ]:
y_wo_std.describe()

In [ ]:
X_train_wo_std, X_test_wo_std, y_train_wo_std, y_test_wo_std = sklearn.model_selection.train_test_split(
    X_wo_std, y_wout_out, test_size=0.2, random_state=12345)

In [ ]:
params.append("Lin. Reg. wout outl std")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X_train_wo_std, X_test_wo_std, y_train_wo_std, y_test_wo_std))
results = pd.DataFrame(res, index=params)

In [ ]:
results

In [ ]:
# the same model with standardized target variable
X_train_wo_std_y, X_test_wo_std_y, y_train_wo_std_y, y_test_wo_std_y = sklearn.model_selection.train_test_split(
    X_wo_std, y_wo_std, test_size=0.2, random_state=12345)

In [ ]:
params.append("Lin. Reg. wout outl std y")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X_train_wo_std_y, X_test_wo_std_y, y_train_wo_std_y, y_test_wo_std_y))
results = pd.DataFrame(res, index=params)

In [ ]:
results

In [ ]:
# MinMax normalization of the dataset
X_wo_norm = (X_wout_out - X_wout_out.min())/(X_wout_out.max() - X_wout_out.min())
y_wo_norm = (y_wout_out - y_wout_out.min())/(y_wout_out.max() - y_wout_out.min())

In [ ]:
X_wo_norm.describe()

In [ ]:
X_train_wo_norm, X_test_wo_norm, y_train_wo_norm, y_test_wo_norm = sklearn.model_selection.train_test_split(
    X_wo_norm, y_wout_out, test_size=0.2, random_state=12345)

In [ ]:
params.append("Lin. Reg. wout outl norm")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X_train_wo_norm, X_test_wo_norm, y_train_wo_norm, y_test_wo_norm))
results = pd.DataFrame(res, index=params)

In [ ]:
results

In [ ]:
# the same model with normalized target variable
# THE BIG TRAP!!!
X_train_wo_norm_y, X_test_wo_norm_y, y_train_wo_norm_y, y_test_wo_norm_y = sklearn.model_selection.train_test_split(
    X_wo_norm, y_wo_norm, test_size=0.2, random_state=12345)

In [ ]:
params.append("Lin. Reg. wout outl norm y")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X_train_wo_norm_y, X_test_wo_norm_y, y_train_wo_norm_y, y_test_wo_norm_y))
results = pd.DataFrame(res, index=params)

In [ ]:
results

## Reduction of the problem dimension

In [ ]:
# let's have a look on variables importance in the dataset without outliers
lm_wo_std = sklearn.linear_model.LinearRegression()
lm_wo_std.fit(X_wo_std, y_wo_std)

In [ ]:
pd.Series(np.abs(lm_wo_std.coef_), index=X_wo_std.columns.to_list()).round(4).sort_values(ascending=False)

In [ ]:
# chlorides is the least important feature, so we will remove it
X_wout_out_chl = X_wout_out.drop(columns=['chlorides']).copy()

In [ ]:
X_wout_out_chl

In [ ]:
X_train_wo_chl, X_test_wo_chl, y_train_wo_chl, y_test_wo_chl = sklearn.model_selection.train_test_split(
    X_wout_out_chl, y_wout_out, test_size=0.2, random_state=12345)

In [ ]:
params.append("Lin. Reg. wout outl chl")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X_train_wo_chl, X_test_wo_chl, y_train_wo_chl, y_test_wo_chl))
results = pd.DataFrame(res, index=params)

In [ ]:
results

In [ ]:
# the residual.sugar and density are strongly correlate
# but the correlation of the residual.sugar with the target value (alcohol) is smaller
# so let's remove the residual.sugar
corr_P_wout_outl

In [ ]:
X_wout_out_res = X_wout_out.drop(columns=['residual.sugar']).copy()

In [ ]:
X_wout_out_res

In [ ]:
X_train_wo_res, X_test_wo_res, y_train_wo_res, y_test_wo_res = sklearn.model_selection.train_test_split(
    X_wout_out_res, y_wout_out, test_size=0.2, random_state=12345)

In [ ]:
params.append("Lin. Reg. wout outl res")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X_train_wo_res, X_test_wo_res, y_train_wo_res, y_test_wo_res))
results = pd.DataFrame(res, index=params)

In [ ]:
# ANOTHER BIG TRAP!!!
results

## The regression model based on polynomials

In [ ]:
# we use the PolynomialFeatures function with grade 2,
# to generate new features that are the product of the base features,
#for example [x1,x2,x3] -> [x1, x2, x3, x1^2, x1x2, x1x3, x2^2, x2x3, x3^2]

import sklearn.preprocessing
polynomial2_feature = sklearn.preprocessing.PolynomialFeatures(degree=2, include_bias=False)
polynomial2_feature.fit_transform(np.array([[2,3,5],[1,2,3]]))

In [ ]:
#we can check the powers of individual variables (we look at the columns)
polynomial2_feature.powers_.T

In [ ]:
# we build a polynomial model transforming the training dataset of predictors X_train_wo
# and the test dataset of predictors X_test_wo
polynomial2 = sklearn.preprocessing.PolynomialFeatures(degree=2, include_bias=False)
X2_wo_train = polynomial2.fit_transform(X_train_wo)
X2_wo_test = polynomial2.fit_transform(X_test_wo)

In [ ]:
#now we have 65 columns
X2_wo_train.shape

In [ ]:
#we check the new model
params.append("Lin. Reg. wout outl Poly")
res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X2_wo_train, X2_wo_test, y_train_wo, y_test_wo))
results = pd.DataFrame(res, index=params)


In [ ]:
results

#### we obtained smaller prediction errors, but the number of model parameters increased significantly

#### we are looking for a balance between the complexity of the model and its quality

In [ ]:
# the choice of variables for the model can be made using the Schwarz criterion (BIC - Bayesian Information Criterion)
# we choose a regression model that minimizes the function
# BIC (MSE_p, p, n) = n * log (MSE_p) + p * log (n)
# MSE_p is calculated for the model built on the basis of p <= d variables
# p * log (n) is a penalty for the complexity of the model

In [ ]:
def BIC(mse, p, n):
    return n*np.log(mse) + p*np.log(n)

In [ ]:
# the number of all possible cases to be considered is in the order of 2^d
# 1. we start with the empty model, BIC = +infinity
# 2. we extend the model with the variable for which BIC is the smallest and
# simultaneously decreases the current BIC value
# if there is no such value, we return the current model
# 3. we repeat the 2nd until exhausted

In [ ]:
def forward_selection(X, y):
    n, m = X.shape
    best_idx = []
    best_free = set(range(m))
    best_fit = np.inf
    res = []
    
    for i in range(0, m):
        cur_idx = -1
        cur_fit = np.inf
        for e in best_free:
            r = sklearn.linear_model.LinearRegression()
            test_idx = best_idx + [e]
            r.fit(X[:, test_idx], y)
            test_fit = BIC(sklearn.metrics.mean_squared_error(y, r.predict(X[:, test_idx])), i+2, n)
            if test_fit < cur_fit: cur_idx, cur_fit = e, test_fit
        if cur_fit > best_fit: break
        
        best_idx, best_fit = best_idx + [cur_idx], cur_fit
        best_free.discard(cur_idx)
        res.append((cur_idx, cur_fit))
    return res

In [ ]:
#we apply the variable selection algorithm to the polynomial transformed dataset

chosen_df = pd.DataFrame(forward_selection(X2_wo_train, y_train_wo), columns=["variable", "BIC"])

chosen_variables = chosen_df["variable"].tolist()

chosen_df["name_of_variable"] =\
[X_wout_out.columns[w>=1].append(X_wout_out.columns[w==2]).str.cat(sep="*") for w in polynomial2.powers_[chosen_variables]]

chosen_df

In [ ]:
chosen_variables

In [ ]:
#and the results
params.append("Lin. Reg. BIC")

res.append(fit_regression(sklearn.linear_model.LinearRegression(),
                          X2_wo_train[:, chosen_variables],
                          X2_wo_test[:, chosen_variables],
                          y_train_wo, y_test_wo))

results = pd.DataFrame(res, index=params)


In [ ]:
results

## Please check the BIC chosen features with cross validation

In [ ]:
# and summary with some figures
plt.figure(figsize=(12,6))

plt.plot(results['RMSE_tr'], label='traning')
plt.plot(results['RMSE_te'], label='test')
plt.legend()
plt.xticks(np.arange(len(results.index.tolist())), results.index.tolist(), rotation=75)

plt.show()

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(results['MAE_tr'], label='traning')
plt.plot(results['MAE_te'], label='test')
plt.legend()
plt.xticks(np.arange(len(results.index.tolist())), results.index.tolist(), rotation=75)

plt.show()

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(results['r_score_tr'], label='traning')
plt.plot(results['r_score_te'], label='test')
plt.legend()
plt.xticks(np.arange(len(results.index.tolist())), results.index.tolist(), rotation=75)

plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

In [ ]:
# cross validation for several models based on dataset without outliers
methods = pd.Series({
    "LinearRegression": sklearn.linear_model.LinearRegression(),
    "ElasticNet": sklearn.linear_model.ElasticNet(),
    "Ridge": sklearn.linear_model.Ridge(),
    "KNN": KNeighborsRegressor(),
    "GP": GaussianProcessRegressor(),
    "RF": RandomForestRegressor(),
    "SVR": SVR()
})

#evaluation function
def eval_function(X_train, X_test, y_train, y_test):
    cv_models = pd.concat([
        pd.Series(fit_regression(alg,
                                 X_train, X_test, y_train, y_test)) for alg in methods], axis=1).T
    cv_models.index = methods.index
    return cv_models

kf = KFold(n_splits=5)

n_folds = 5

results_cv = [eval_function(X_wout_out.iloc[train,:],
                            X_wout_out.iloc[test,:],
                            y_wout_out.iloc[train],
                            y_wout_out.iloc[test]) for train, test in kf.split(X_wout_out)]

sum(results_cv)/n_folds

# :) We have to discuss these results.

In [ ]:
results_cv = [eval_function(X.iloc[train,:],
                            X.iloc[test,:],
                            y.iloc[train],
                            y.iloc[test]) for train, test in kf.split(X)]

sum(results_cv)/n_folds